In [ ]:
# Notebook to display data for selected highway links flow, V/C, and speeds using the folum library.
# 
# Approach: Load spatial (shapefile) and tabular data into a geodataframe,
#           export the geodataframe to a GeoJSON file,
#           and then load the GeoJSON file into a folium map.
#
# Tabular data was previously calculated by highway_links_report.ipynb, and saved in CSV format.
#
import openmatrix as omx
import numpy as np
import pandas as pd
import geopandas as gp
from io import StringIO
import matplotlib.pyplot as plt
import jenkspy
import folium
import bokeh
import geoviews
import hvplot.pandas

In [ ]:
%matplotlib notebook

In [ ]:
# Directory in which user's output CSV report data was saved - it will now be our *input* directory
my_sandbox_dir = r'S:/my_modx_output_dir/'

In [ ]:
# Name of CSV file with volume, V/C, and speed data for links with specified functional clas(ses)
# generated by the notebook "highway_links_report_by_functional_class.ipynb."
# This will now be our *input* CSV file.
#
csv_fn = 'functional_class_report_base_scenario.csv'

In [ ]:
# Fully-qualified pathname to CSV file
fq_csv_fn = my_sandbox_dir + csv_fn

In [ ]:
links_data_df = pd.read_csv(fq_csv_fn, delimiter=',')

In [ ]:
links_data_df

In [ ]:
list(links_data_df.columns)

In [ ]:
# List of the IDs for the model network links for which data is reported in the input CSV file
links_list = links_data_df['ID1'].to_list()

In [ ]:
# Directory in which the spatial data for the model network links is stored (both shapefile and GeoJSON formats)
links_spatial_data_dir = r'G:/Data_Resources/modx/statewide_links_shapefile/'

In [ ]:
# Load the links shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
links_shapefile_fn = 'Statewide_Links_2018_BK_EPSG4326.shp'
fq_links_shapefile_fn = links_spatial_data_dir + links_shapefile_fn
links_gdf = gp.read_file(fq_links_shapefile_fn)
links_gdf.set_index("ID")

In [ ]:
# Filter the links geodataframe to only the links of interest
filtered_links_gdf = links_gdf[links_gdf['ID'].isin(links_list)] 

In [ ]:
filtered_links_gdf

In [ ]:
# Join the geo-data frame for the links with the "links_data_df", which contains the computed data about these links
join_df = filtered_links_gdf.join(links_data_df.set_index("ID1"), on="ID")

In [ ]:
join_df

In [ ]:
# Return the bounding box of all the features in a geo-dataframe.
# The bounding box is returned as a dictionary with the following keys: { 'minx', 'miny', 'maxx', 'maxy'}.
#
def bbox_of_gdf(gdf):
    bounds_tuples = gdf['geometry'].map(lambda x: x.bounds)
    bounds_dicts = []
    for t in bounds_tuples:
        temp = { 'minx' : t[0], 'miny' : t[1], 'maxx' : t[2], 'maxy' : t[3] }
        bounds_dicts.append(temp)
    # end_for
    bounds_df = pd.DataFrame(bounds_dicts)
    minx = bounds_df['minx'].min()
    miny = bounds_df['miny'].min()
    maxx = bounds_df['maxx'].max()
    maxy = bounds_df['maxy'].max()
    retval = { 'minx' : minx, 'miny' : miny, 'maxx' : maxx, 'maxy' : maxy }
    return retval
# end_def bbox_of_gdf()

# Given a dictonary of the form  'minx', 'miny', 'maxx', 'maxy'} representing a geographic bounding box,
# return the center point as a dictionary with the keys { 'x' , 'y' }.
def center_of_bbox(bbox):
    center_x = bbox['minx'] + (bbox['maxx'] - bbox['minx']) / 2
    center_y = bbox['miny'] + (bbox['maxy'] - bbox['miny']) / 2
    retval = { 'x' : center_x, 'y' : center_y }
    return retval
# end_def center_of_bbox()

In [ ]:
# Get the bounding box of the selected links, and the center point of that bounding box
bbox = bbox_of_gdf(join_df)
center_pt = center_of_bbox(bbox)

In [ ]:
# Directory containing miscellaneous reference data
misc_reference_data_dir = r'G:/Data_Resources/modx/misc_reference_data/'

In [ ]:
# Load the MassGIS TOWNS_POLYM shapefile into a geopandas dataframe 
# NOTE: This version of the shapefile is in EPSG4326, i.e., WGS84
towns_shapefile_fn = 'towns_polym_EPSG4326.shp'
fq_towns_shapefile_fn = misc_reference_data_dir + towns_shapefile_fn
towns_gdf = gp.read_file(fq_towns_shapefile_fn)
towns_gdf.set_index("town_id")

In [ ]:
# Export the geo-dataframe to GeoJSON format, so it can be used with the folium library
out_geojson_fn = my_sandbox_dir + 'temp_geojson.geojson'
join_df.to_file(out_geojson_fn, driver='GeoJSON')

In [ ]:
# Make a static map of speed during the AM period overlayed on the towns layer
base = towns_gdf.plot(color='white', edgecolor='black')
join_df.plot("Speed_am", ax=base, figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.xlim((bbox['minx'], bbox['maxx']))
plt.ylim((bbox['miny'], bbox['maxy']))
plt.title('Speed in AM')
plt.show()

In [ ]:
# Render an interactive folium map of AM speed
# 
model_region_center = [42.27, -71.73]
m = folium.Map(location=model_region_center, zoom_start=8)
links_geojson = open(out_geojson_fn).read()
#
# Color scale source: https://colorbrewer2.org/#type=diverging&scheme=RdYlGn&n=6 (inverted)
def speed_colorscale(speed):
    if speed == None:
        retval = '#000000'
    elif (speed > 50.0):
        retval = '#1a9850'
    elif (speed > 40.0):
        retval = '#91cf60'
    elif (speed > 30.0):
        retval = '#d9ef8b'
    elif (speed > 20.0):
        retval = '#fee08b'
    elif (speed > 10.0):
        retval = '#fc8d59'
    else:
        retval = '#d73027'
    #
    return retval
#
def my_style_function(feature):
    speed = feature['properties']['Speed_am']
    return {
        'opacity': 1.0,
        'weight' : 5.0,
        'color': speed_colorscale(speed)
    }
#
folium.GeoJson(links_geojson,
               style_function=my_style_function).add_to(m)

#
m

In [ ]:
# Make a static map of total daily flow (volume) during the AM period
base = towns_gdf.plot(color='white', edgecolor='black')
join_df.plot("Tot_Flow_daily", ax=base, figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.xlim((bbox['minx'], bbox['maxx']))
plt.ylim((bbox['miny'], bbox['maxy']))
plt.title('Daily Total Flow (volume)')
plt.show()

In [ ]:
# Make an interactive folium map of total daily flow (volume) during the AM period
# 
model_region_center = [42.27, -71.73]
m = folium.Map(location=model_region_center, zoom_start=8)
links_geojson = open(out_geojson_fn).read()
#
# Colorscale source = https://colorbrewer2.org/#type=sequential&scheme=Reds&n=7 (inverted)
def flow_colorscale(flow):
    if flow == None:
        retval = '#00000'
    elif (flow > 120000.0):
        retval = '#99000d'
    elif (flow > 100000.0):
        retval = '#cb181d'
    elif (flow > 80000.0):
        retval = '#ef3b2c'
    elif (flow > 60000.0):
        retval = '#fb6a4a'
    elif (flow > 40000.0):
        retval = '#fc9272'
    elif (flow > 20000.0):
        retval = '#fcbba1'
    else:
        retval = '#fee5d9'
    #
    return retval
#
def my_style_function(feature):
    flow = feature['properties']['Tot_Flow_daily']
    return {
        'opacity': 1.0,
        'weight' : 5.0,
        'color': flow_colorscale(flow)
    }
#
folium.GeoJson(links_geojson,
               style_function=my_style_function).add_to(m)

#
m

In [ ]:
# Make a static map of the volume-to-capacity ratio during the AM period
base = towns_gdf.plot(color='white', edgecolor='black')
join_df.plot("VOC_am", ax=base, figsize=(10.0,8.0), cmap='plasma', legend=True)
plt.xlim((bbox['minx'], bbox['maxx']))
plt.ylim((bbox['miny'], bbox['maxy']))
plt.title('Volume/Capacity Ratio')
plt.show()

In [ ]:
# Make an interactive folium map of the volume-to-capacity ratio during the AM period
# 
model_region_center = [42.27, -71.73]
m = folium.Map(location=model_region_center, zoom_start=8)
links_geojson = open(out_geojson_fn).read()
# 
# # Colorscale source = https://colorbrewer2.org/#type=sequential&scheme=Blues&n=4 (inverted)
def voc_colorscale(voc):
    if voc == None:
        retval = '#000000'
    elif (voc > 1.5):
        retval = '#2171b5'
    elif (voc > 1.0):
        retval = '#6baed6'
    elif (voc > 0.5):
        retval = '#bdd7e7'
    else:
        retval = '#eff3ff'
    #
    return retval
#
def my_style_function(feature):
    voc = feature['properties']['VOC_am']
    return {
        'opacity': 1.0,
        'weight' : 5.0,
        'color': voc_colorscale(voc)
    }
#
folium.GeoJson(links_geojson,
               style_function=my_style_function).add_to(m)

#
m